# Tree Classification with Deep Neural Network / Random Forest

## Dataset Preprocessing

The dataset preprocessing includes loading the raw npy dataset that has been created with the convertToNpy.py script. The loaded data is then separated into the X (input array) and Y (ground truth label). The input array is then normalized using the min-max scaler which is implemented in the NormalizeMatrix function. Then the data is split into training, validation, and test dataset.

In [85]:
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split
from TreeDataset import TreeDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import math
from torch.nn import init
import torchvision.models as models

#convert image matrix to a numpy array and feed it into dataset

image = np.load("assets/data_without_label.npy")
labels = np.load("assets/labels.npy",allow_pickle = True)

print(image.shape)
print(labels.shape)

le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

print(np.count_nonzero(encoded_labels == 4))

dataset = TreeDataset(image, encoded_labels)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
torch.cuda.get_device_name()

(35682, 10, 5, 5)
(35682,)
8412


'NVIDIA GeForce GTX 1050'

In [86]:
# Define the ratio of the split (e.g., 80% for training, 20% for testing)
train_val_ratio = 0.8
test_ratio = 1 - train_val_ratio
train_ratio = 0.8

# Calculate the number of samples for each split
num_samples = len(dataset)
train_val_size = int(train_val_ratio * num_samples)
test_size = num_samples - train_val_size

train_size = int(train_ratio * train_val_size)
val_size = train_val_size - train_size

print(train_val_size)

generator = torch.Generator().manual_seed(42)
val_generator = torch.Generator().manual_seed(18)

# Randomly split the dataset into training and test sets
train_val_dataset, test_dataset = random_split(dataset, [train_val_size, test_size],generator=generator)
      
train_dataset, val_dataset = random_split(train_val_dataset,[train_size, val_size], generator=val_generator )

# Create data loaders for the training and test sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

28545


In [87]:
# To be solved??
# Current data shape (num_data, num_channels, pixel_row, pixel_col)

from models import ModifiedResNet, MultispectralClassifier, MyNet, ResidualBlock

# cnnModel = ModifiedResNet(ResidualBlock, [3, 4, 6, 3])

cnnModel = MultispectralClassifier(10)

In [88]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnnModel.parameters(), lr=0.01)

In [93]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.float())
        loss = loss_fn(pred, y.long())

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # Calculate Loss
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in valid_loader:
        # Transfer Data to GPU if available
        data, labels = data.float(), labels.long()
        # Forward Pass
        target = model(data.float())
        # Find the Loss
        valid_loss = loss_fn(target,labels)
        if batch % 100 == 0:
            valid_loss = valid_loss.item()
            print(f"Valid_loss: {valid_loss:>7f} [{current:>5d}/{size:>5d}]")


    
        



def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.float())
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [94]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, cnnModel, loss_fn, optimizer)
    test_loop(test_loader, cnnModel, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.040650  [   32/22836]
loss: 2.188697  [ 3232/22836]
loss: 2.072292  [ 6432/22836]
loss: 2.083125  [ 9632/22836]
loss: 2.035035  [12832/22836]
loss: 2.085088  [16032/22836]
loss: 2.072678  [19232/22836]
loss: 2.267495  [22432/22836]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.094500 

Epoch 2
-------------------------------
loss: 1.962896  [   32/22836]
loss: 2.082603  [ 3232/22836]
loss: 2.166827  [ 6432/22836]
loss: 1.970408  [ 9632/22836]
loss: 2.055088  [12832/22836]
loss: 2.071980  [16032/22836]
loss: 1.933033  [19232/22836]
loss: 2.010544  [22432/22836]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.095809 

Epoch 3
-------------------------------
loss: 2.159023  [   32/22836]
loss: 2.155641  [ 3232/22836]
loss: 2.006169  [ 6432/22836]
loss: 1.996036  [ 9632/22836]
loss: 2.055849  [12832/22836]
loss: 1.999364  [16032/22836]
loss: 1.931964  [19232/22836]
loss: 2.049407  [22432/22836]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.092902 

Epoc

In [98]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
label_collector = []
prediction_collector = []

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = cnnModel(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        label_collector.append(labels)
        prediction_collector.append(predicted)
print(f'Accuracy of the network on the test images: {100 * correct // total} %')

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


#cm = confusion_matrix(prediction_collector, label_collector )
#ConfusionMatrixDisplay(cm).plot()



Accuracy of the network on the test images: 23 %


In [99]:
inputs, classes = next(iter(test_loader))

output = cnnModel(inputs.float())
print(output)

_, predicted = torch.max(output.data, 1)

print(predicted==classes)

tensor([[-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -0.0168,  0.4432, -0.9538,  0.8621,  0.7535, -1.0501, -0.7594,
         -0.6650, -0.3778],
        [-0.3199, -

In [27]:


torch.save(cnnModel.state_dict(),"D:\Programming\Python\machinelearning\DSEO\MultispectralClassifier.pth")

In [29]:
from torchinfo import summary

summary(cnnModel, input_size=(batch_size, 10, 5, 5))


Layer (type:depth-idx)                   Output Shape              Param #
MultispectralClassifier                  [32, 10]                  --
├─Sequential: 1-1                        [32, 64, 1, 1]            --
│    └─Conv2d: 2-1                       [32, 32, 5, 5]            2,912
│    └─ReLU: 2-2                         [32, 32, 5, 5]            --
│    └─MaxPool2d: 2-3                    [32, 32, 2, 2]            --
│    └─Conv2d: 2-4                       [32, 64, 2, 2]            18,496
│    └─ReLU: 2-5                         [32, 64, 2, 2]            --
│    └─MaxPool2d: 2-6                    [32, 64, 1, 1]            --
├─Sequential: 1-2                        [32, 10]                  --
│    └─Linear: 2-7                       [32, 128]                 8,320
│    └─ReLU: 2-8                         [32, 128]                 --
│    └─Linear: 2-9                       [32, 10]                  1,290
Total params: 31,018
Trainable params: 31,018
Non-trainable params: 0
To